In [114]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import sqlite3

In [115]:

url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [116]:
movies = soup.select('td.titleColumn')
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value')
for b in soup.select('td.posterColumn span[name=ir]')]


In [117]:
import requests
from bs4 import BeautifulSoup

def get_topics_page():
    # TODO - add comments
    topic_url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=000&ref_=adv_next'
    response=requests.get(topic_url)
    # check successfull response
    if response.status_code != 200:
        raise Exception(f'Failed to load page {topic_url}')
    # Parse using BeautifulSoup
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

In [118]:
doc=get_topics_page()

In [119]:
def get_movie_titles(doc):
    
    selection_class="lister-item-header"
    movie_title_tags=doc.find_all('h3',{'class':selection_class})
    movie_titles=[]

    for tag in movie_title_tags:
        title = tag.find('a').text
        movie_titles.append(title)
        
        
    return movie_titles

In [120]:
titles = get_movie_titles(doc)

In [121]:
def get_movie_year(doc):
    year_selector = "lister-item-year text-muted unbold"           
    movie_year_tags=doc.find_all('span',{'class':year_selector})
    movie_year_tagss=[]
    for tag in movie_year_tags:
        movie_year_tagss.append(tag.get_text().strip()[1:5])
    return movie_year_tagss

In [122]:
years = get_movie_year(doc)

In [123]:
def get_movie_url(doc):
    url_selector="lister-item-header"           
    movie_url_tags=doc.find_all('h3',{'class':url_selector})
    movie_url_tagss=[]
    base_url = 'https://www.imdb.com/'
    for tag in movie_url_tags:
        movie_url_tagss.append('https://www.imdb.com/' + tag.find('a')['href'])
    return movie_url_tagss

In [124]:
urls = get_movie_url(doc)

In [160]:
images=[]

In [161]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

movie_data = soup.findAll('div', attrs={'class': 'lister-item mode-advanced'})

for store in movie_data:
    imageDiv = store.find('div', {'class': 'lister-item-image float-left'})
    img = imageDiv.a.img["loadlate"]
    images.append(str(img)[:-24]+".jpg")

In [162]:
images

['https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_.jpg',
 'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_.jpg',
 'https://m.media-amazon.com/images/M/MV5BMTMxNTMwODM0NF5BMl5BanBnXkFtZTcwODAyMTk2Mw@@._V1_.jpg',
 'https://m.media-amazon.com/images/M/MV5BNzA5ZDNlZWMtM2NhNS00NDJjLTk4NDItYTRmY2EwMWZlMTY3XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_.jpg',
 'https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg',
 'https://m.media-amazon.com/images/M/MV5BMWMwMGQzZTItY2JlNC00OWZiLWIyMDctNDk2ZDQ2YjRjMWQ0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_.jpg',
 'https://m.media-amazon.com/images/M/MV5BMWU4N2FjNzYtNTVkNC00NzQ0LTg0MjAtYTJlMjFhNGUxZDFmXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_.jpg',
 'https://m.media-amazon.com/images/M/MV5BMDhhMjBlMWYtMDVlMy00ZjM3LTg3MTUtZDg0NjA0YzcxOGY4XkEyXkFqcGdeQXVyMTE0MzY0NjE1._V1

In [163]:
# dictionary of lists 
dict = {'movie_link': urls, 'movie_name': titles, 'movie_year': years, "movie_mainpage":1,"movie_image":images,"movie_category":"This category of this movie will be defined soon!"} 
    
df = pd.DataFrame(dict)

In [164]:
df.head()

,movie_link,movie_name,movie_year,movie_mainpage,movie_image,movie_category
0,https://www.imdb.com//title/tt0111161/,The Shawshank Redemption,1994,1,https://m.media-amazon.com/images/M/MV5BMDFkYT...,This category of this movie will be defined soon!
1,https://www.imdb.com//title/tt0068646/,The Godfather,1972,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,This category of this movie will be defined soon!
2,https://www.imdb.com//title/tt0468569/,The Dark Knight,2008,1,https://m.media-amazon.com/images/M/MV5BMTMxNT...,This category of this movie will be defined soon!
3,https://www.imdb.com//title/tt0167260/,The Lord of the Rings: The Return of the King,2003,1,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,This category of this movie will be defined soon!
4,https://www.imdb.com//title/tt0108052/,Schindler's List,1993,1,https://m.media-amazon.com/images/M/MV5BNDE4OT...,This category of this movie will be defined soon!


# Insert Data to DB

In [165]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [166]:
path = "/Users/berkayalan/Desktop/Data Science/10 - Python Backend/Django/movieapp"

In [167]:
conn = sqlite3.connect(f'/{path}/db.sqlite3')

In [168]:
c = conn.cursor()

In [169]:
df.to_sql(name="movies_movie", con=conn, if_exists='append', index=False)

100